# **ETL pipeline project to extract top banks data and sort top 10 banks on their market cap in USD and then transform to GBP, EUR and INR as well, in accordance with the exchange rate information**

Following is a demonstration of:


*   Using website **api** and **python requests library** to **get** information from *archive.org* website
*   Use **BeautifulSoup** to parse the file
*   Then **transform** the data to the required format
*   Then save it as **CSV** and **database**

In [ ]:
# Importing required libraries and packages

import pandas as pd
import numpy as np
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import sqlite3
import lxml

In [ ]:
# Preliminaries
def preliminaries():
  global url, exchange_rate_url, table_attribs, final_table_attribs, csv_path, db_name, db_table
  url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
  exchange_rate_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
  table_attribs = ["Name", "MC_USD_Billion"]
  final_table_attribs = ["Name", "MC_USD_Billion", "MC_GBP_Billion", "MC_EUR_Billion", "MC_INR_Billion"]
  csv_path = './Largest_banks_data.csv'
  db_name = 'Banks.db'
  db_table = 'Largest_banks'


In [ ]:
# Logging process

def log_progress(message):
  timestamp_format = '%Y-%h-%d-%H:%M:%S'
  now = datetime.now()
  timestamp = now.strftime(timestamp_format)
  with open('logfile.txt', 'a') as f:
    f.write(timestamp + ':' + message + '\n')


In [ ]:
# Extraction of data from url

def extract(url):
  page = requests.get(url).text
  data = BeautifulSoup(page, 'html.parser')
  df = pd.DataFrame(columns=table_attribs)
  tables = data.find_all('tbody')
  rows = tables[0].find_all('tr')

  for row in rows:
    col = row.find_all('td')
    if len(col) != 0:
      links = col[1].find_all('a')
      if len(links) != 0:
        data_dict = {'Name' : links[-1].text,
                     'MC_USD_Billion' : col[2].text}
        df1 = pd.DataFrame(data_dict, index=[0])
        df = pd.concat([df, df1], ignore_index=True)
  return df


In [ ]:
# Transformation of the data

def transform(df):
  exchange_rate = pd.read_csv(exchange_rate_url, index_col=0)
  df["MC_GBP_Billion"] = df["MC_USD_Billion"].astype(float) * exchange_rate.loc["GBP", "Rate"]
  df["MC_EUR_Billion"] = df["MC_USD_Billion"].astype(float) * exchange_rate.loc["EUR", "Rate"]
  df["MC_INR_Billion"] = df["MC_USD_Billion"].astype(float) * exchange_rate.loc["INR", "Rate"]
  return df

In [ ]:
# Loading data

def to_csv(df, csv_path):
  df.to_csv(csv_path)

def to_sql(df, db_name, db_table):
  connection = sqlite3.connect(db_name)
  df.to_sql(db_table, connection, if_exists='replace', index=False)

In [ ]:
# Querying db

def run_query(query_statement, db_name):
  connection = sqlite3.connect(db_name)
  query_output = pd.read_sql(query_statement, connection)
  print(query_output)
  return query_output

In [ ]:
# Function calls

preliminaries()
log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url)
log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df)
log_progress('Data transformation complete. Initiating loading process')
to_csv(df, csv_path)
to_sql(df, db_name, db_table)
log_progress('Data saved to CSV file and into Banks.db')
query_statement_1 = 'SELECT * FROM Largest_banks'
query_statement_2 = 'SELECT AVG(MC_GBP_Billion) FROM Largest_banks'
query_statement_3 = 'SELECT Name from Largest_banks LIMIT 5'
run_query(query_statement_1, db_name)
run_query(query_statement_2, db_name)
run_query(query_statement_3, db_name)
log_progress('Process Complete.')

                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank
